# Demonstration
* In this Jupyter Notebook we show using MockClients how the translation process will work.
* It is possible to run the scripts on Terminal but we use Jupyter Notebooks to document exactly what the author used to run the code.

In [1]:
%run proc0.py run -m gpt -d opus-100

INFO: 2025-05-05 12:04:10 - [🚀]: Selected task for opus-100 - gpt
INFO: 2025-05-05 12:04:10 - [🏁]: Starting task b25b0219-5e76-40d6-98e3-b11cff4af75f on commit fa3a65b
ERROR: 2025-05-05 12:04:10 - [⚠️]: Error MockError
INFO: 2025-05-05 12:04:10 - [⏲️]: Retrying de-en...
INFO: 2025-05-05 12:04:10 - [❌]: Translated 200 sents for de-en but rejected
INFO: 2025-05-05 12:04:10 - [⏲️]: Retrying de-en...
INFO: 2025-05-05 12:04:10 - [✔️]: Translated 400 sents for de-en
INFO: 2025-05-05 12:04:11 - [✔️]: Translated 400 sents for en-de
INFO: 2025-05-05 12:04:11 - [✔️]: Translated 400 sents for da-en
INFO: 2025-05-05 12:04:11 - [✔️]: Translated 400 sents for en-da
INFO: 2025-05-05 12:04:11 - [✔️]: Translated 400 sents for el-en
INFO: 2025-05-05 12:04:11 - [✔️]: Translated 400 sents for en-el
INFO: 2025-05-05 12:04:12 - [✔️]: Translated 400 sents for pt-en
INFO: 2025-05-05 12:04:12 - [✔️]: Translated 400 sents for en-pt
INFO: 2025-05-05 12:04:12 - [✔️]: Translated 400 sents for sv-en
INFO: 2025-05-0

* Logs observable when running the task using the `proc0.py` script are brief
* The same logs are also stored in a `.log` file that is more detailed in case of errors
    * This process is not run on a server nor requires deployments, hence it makes more sense to have the real-time logs brief / readable, whereas the stored logs are more detailed for debugging purposes
    * Since we use Jupyter Notebook, it is possible to 'store' both types of logs (given the notebook is not re-run by accident...)
 

In [2]:
!cat proc0.log | head -n 18

INFO: 2025-05-05 12:04:10 - [🚀]: Selected task for opus-100 - gpt
INFO: 2025-05-05 12:04:10 - [🏁]: Starting task b25b0219-5e76-40d6-98e3-b11cff4af75f on commit fa3a65b
ERROR: 2025-05-05 12:04:10 - [⚠️]: Error MockError
DEBUG: 2025-05-05 12:04:10 - Traceback:
Traceback (most recent call last):
  File "C:\Files\UZH\Semester_6\BA_Thesis\BA_Repo\scripts\task.py", line 173, in run
    mt_sents = self.client.translate_and_store_document(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Files\UZH\Semester_6\BA_Thesis\BA_Repo\scripts\translators.py", line 54, in translate_and_store_document
    out_text = self.translate_document(
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Files\UZH\Semester_6\BA_Thesis\BA_Repo\scripts\translators.py", line 271, in translate_document
    out_text = self.encrypt(in_text, error_pair=(src_lang, tgt_lang))
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Files\UZH\Semester_6\BA_Thesis\BA_Repo\scripts\transl

* We can reproduce the simpler logs using `grep`
    * On Windows we need to use double quotations for the Regex, for Linux/Mac single quotes would work too 

In [3]:
!grep -P "(INFO:|ERROR:)" proc0.log | head -n 6

INFO: 2025-05-05 12:04:10 - [🚀]: Selected task for opus-100 - gpt
INFO: 2025-05-05 12:04:10 - [🏁]: Starting task b25b0219-5e76-40d6-98e3-b11cff4af75f on commit fa3a65b
ERROR: 2025-05-05 12:04:10 - [⚠️]: Error MockError
INFO: 2025-05-05 12:04:10 - [⏲️]: Retrying de-en...
INFO: 2025-05-05 12:04:10 - [❌]: Translated 200 sents for de-en but rejected
INFO: 2025-05-05 12:04:10 - [⏲️]: Retrying de-en...


* In addition to task specific logs, we also log more details per translation in a JSONL file, for analysis purposes

In [4]:
!cat tmp/proc0.jsonl | head -n 5

{"src_lang": "de", "tgt_lang": "en", "start": 1746439450.552189, "in_lines": 400, "in_sents": 444, "in_chars": 32731, "in_tokens": 8295, "end": 1746439450.6247172, "out_chars": 15366, "out_lines": 200, "out_sents": 237, "out_tokens": 7368, "id": "b25b0219-5e76-40d6-98e3-b11cff4af75f-0002", "translator": "gpt", "dataset": "Helsinki-NLP/opus-100", "verdict": "rejected"}
{"src_lang": "de", "tgt_lang": "en", "start": 1746439450.687235, "in_lines": 400, "in_sents": 444, "in_chars": 32731, "in_tokens": 8295, "end": 1746439450.7562985, "out_chars": 32731, "out_lines": 400, "out_sents": 478, "out_tokens": 15578, "id": "b25b0219-5e76-40d6-98e3-b11cff4af75f-0003", "translator": "gpt", "dataset": "Helsinki-NLP/opus-100", "verdict": "accepted"}
{"src_lang": "en", "tgt_lang": "de", "start": 1746439450.9567137, "in_lines": 400, "in_sents": 446, "in_chars": 29414, "in_tokens": 6994, "end": 1746439451.0415869, "out_chars": 29414, "out_lines": 400, "out_sents": 449, "out_tokens": 13305, "id": "b25b0219

* The translation logs are linked to the task, observe of the translation log ids use the task id as a prefix.

In [5]:
!cat tmp/proc0/opus-100/gpt/task.json

{"task_id": "b25b0219-5e76-40d6-98e3-b11cff4af75f", "git_hash": "fa3a65b", "dataset": "Helsinki-NLP/opus-100", "num_of_sents": 400, "split": "test[:500]", "translator": "gpt", "acceptable_range": [360, 480], "timestamp": "2025-05-05T12:04:15.011240+02:00", "manual_retry": false, "duration": 4.571102619171143}


## Manual Retry
* In cases were a task, even if automatic retry is accounted for, fails to deliver translations that are desired, we need to run the task again. 
* For documentation purposes, we want to link these 'faulty' translations to ones the task delievered before.
* If it is 'faulty' because we never get one due to errors, in such cases we do not log any translations, we still specify a reason but no id
* If it is 'faulty' because we automatically rejected the ones that arrived, we link it to the newest one.
    * Automatic rejection of translations is limited to avoid paying too much
    * Rejection reason is mainly receiving too much or too little output translations. All rejected translations are still stored and have an id. 

* In the following we define a scenario where En-Fr and Fr-En will not be translated successfully because:
    * En-Fr was rejected three times automatically
    * Fr-En failed three times  

In [6]:
pairs = [('en', 'de'), ('en', 'fr'), ('de', 'en'), ('fr', 'en')]
scenario = [0, 1, 1, 1, 0, 2, 2, 2]
verdicts = ['accepted', 'rejected', 'rejected', 'rejected', 'accepted']

In [7]:
from scripts.data_management import Opus100Manager
from scripts.task import TranslationTask
from scripts.translators import MockClient
from scripts.logger import TranslationLogger
from os.path import join
from io import StringIO
main_folder = 'tmp'
sub_folder = join(main_folder, 'proc0')

dm = Opus100Manager()
logfile = StringIO()
logger = TranslationLogger(logfile=logfile)
cli = MockClient(logger=logger, model='mock', scenario=scenario)
task = TranslationTask(
    target_pairs=pairs,
    dm=dm,
    client=cli,
    logger=logger,
    mt_folder=sub_folder,
    num_of_sents=400,
    max_retries=2,
    retry_delay=0
)

In [8]:
task.run()

INFO: 2025-05-05 12:04:15 - [🏁]: Starting task 982e247d-4801-4a96-a8f6-2274f3c53cf0 on commit fa3a65b
INFO: 2025-05-05 12:04:15 - [✔️]: Translated 400 sents for en-de
INFO: 2025-05-05 12:04:15 - [❌]: Translated 200 sents for en-fr but rejected
INFO: 2025-05-05 12:04:15 - [⏲️]: Retrying en-fr...
INFO: 2025-05-05 12:04:15 - [❌]: Translated 200 sents for en-fr but rejected
INFO: 2025-05-05 12:04:15 - [⏲️]: Retrying en-fr...
INFO: 2025-05-05 12:04:16 - [❌]: Translated 200 sents for en-fr but rejected
INFO: 2025-05-05 12:04:16 - [⏩]: Failed 2 times, skipping en-fr...
INFO: 2025-05-05 12:04:16 - [✔️]: Translated 400 sents for de-en
ERROR: 2025-05-05 12:04:16 - [⚠️]: Error MockError
INFO: 2025-05-05 12:04:16 - [⏲️]: Retrying fr-en...
ERROR: 2025-05-05 12:04:16 - [⚠️]: Error MockError
INFO: 2025-05-05 12:04:16 - [⏲️]: Retrying fr-en...
ERROR: 2025-05-05 12:04:16 - [⚠️]: Error MockError
INFO: 2025-05-05 12:04:16 - [⏩]: Failed 2 times, skipping fr-en...
INFO: 2025-05-05 12:04:16 - [🏁]: Task took

In [9]:
import json
log_data = [json.loads(ln) for ln in logfile.getvalue().splitlines()]
[log['verdict'] for log in log_data] == verdicts


True

* Based on these results, we decide to retry En-Fr and Fr-En.
* During analysis, we also noticed that De-En has not been translated at all, BLEU score single digits, src-text was returned, so we retry that as well.

In [10]:
log_ids = [log['id'] for log in log_data if (log['src_lang'] == 'en' and log['tgt_lang'] == 'fr') or (log['src_lang'] == 'de' and log['tgt_lang'] == 'en')]
log_ids

['982e247d-4801-4a96-a8f6-2274f3c53cf0-0002',
 '982e247d-4801-4a96-a8f6-2274f3c53cf0-0003',
 '982e247d-4801-4a96-a8f6-2274f3c53cf0-0004',
 '982e247d-4801-4a96-a8f6-2274f3c53cf0-0005']

In [11]:
for log in log_data:
    if log['id'] == log_ids[-2]:
        print(log['src_lang'], log['tgt_lang'], log['id'])
    if log['id'] == log_ids[-1]:
        print(log['src_lang'], log['tgt_lang'], log['id'])



en fr 982e247d-4801-4a96-a8f6-2274f3c53cf0-0004
de en 982e247d-4801-4a96-a8f6-2274f3c53cf0-0005


In [12]:
from scripts.logger import RetryLog
selected_ids = [log_ids[-2], log_ids[-1], None]
target_pairs = [('en', 'fr'), ('de', 'en'), ('fr', 'en')]
reasons = ['no accepted translation yet', 'returned src text', 'no translation received yet']
retry_log = RetryLog(pairs=target_pairs, log_ids=selected_ids, reasons=reasons)
new_logger = TranslationLogger(logfile=logfile, retry_log=retry_log)
cli = MockClient(logger=new_logger)
sub_folder = join(main_folder, 'tmp2')

task = TranslationTask(
    target_pairs=target_pairs,
    dm=dm,
    client=cli,
    logger=new_logger,
    mt_folder=sub_folder,
    num_of_sents=400,
    manual_retry=True,
    max_retries=1,
    retry_delay=0
)

task.run()

INFO: 2025-05-05 12:04:16 - [🏁]: Starting task 7533f0f3-c142-44ef-aa84-37e7cd21cb5a on commit fa3a65b
INFO: 2025-05-05 12:04:17 - [✔️]: Translated 400 sents for en-fr
INFO: 2025-05-05 12:04:17 - [✔️]: Translated 400 sents for de-en
INFO: 2025-05-05 12:04:17 - [✔️]: Translated 400 sents for fr-en
INFO: 2025-05-05 12:04:17 - [🏁]: Task took 0.82s


In [13]:
log_data = [json.loads(ln) for ln in logfile.getvalue().splitlines()]
for log in log_data[-3:]:
    print(log['src_lang'], log['tgt_lang'], log['manual_retry'])

en fr {'prev_id': '982e247d-4801-4a96-a8f6-2274f3c53cf0-0004', 'reason': 'no accepted translation yet'}
de en {'prev_id': '982e247d-4801-4a96-a8f6-2274f3c53cf0-0005', 'reason': 'returned src text'}
fr en {'prev_id': None, 'reason': 'no translation received yet'}


In [14]:
!rm -rf tmp